<a href="https://colab.research.google.com/github/afvallejo/snPATHOSeq/blob/main/FRP_validation_using_LnCaP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>